In [1]:
from cloudvolume import CloudVolume
import numpy as np
import os
import random
import requests
import tifffile
import matplotlib.pyplot as plt
import fsspec
import zarr
import dask.array as da
import pandas as pd
import xmltodict

In [2]:
XML_EMPIAR_URL = f"https://ftp.ebi.ac.uk/pub/databases/emtest/empiar/headers/11759.xml"
xml_resp = requests.get(XML_EMPIAR_URL)

In [3]:
with open("empiar_meta_11759.xml", "wb") as f:
    f.write(xml_resp.content)

In [4]:
with open("empiar_meta_11759.xml", 'rb') as f:
    xml_empiar = xmltodict.parse(f.read())

In [9]:
xml_empiar.keys()

dict_keys(['entry'])

In [10]:
def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                if isinstance(item, dict):
                    items.extend(flatten_dict(item, f"{new_key}[{i}]", sep=sep).items())
                else:
                    items.append((f"{new_key}[{i}]", item))
        else:
            items.append((new_key, v))
    return dict(items)

In [11]:
meta_flat = flatten_dict(xml_empiar['entry'])

In [ ]:
meta_df = pd.DataFrame(meta_flat.items(), columns=['Field', 'value'])
meta_df.to_csv("EMPIAR/empiar_meta_11759.csv", index=False)

In [15]:
print(meta_df.to_markdown(index=False))

| Field                                                                                 | value                                                                                                                                                            |
|:--------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| @xmlns                                                                                | http://pdbe.org/empiar                                                                                                                                           |
| @xmlns:xsi                                                                            | http://www.w3.org/2001/XMLSchema-instance                                                                                                              